In [1]:
from datetime import datetime
# importing moules
import os
import pandas as pd
import matplotlib
import numpy as np
from re import search
import matplotlib.pyplot as plt
from datetime import date
import seaborn as sns
import math

In [2]:
all_encounters = pd.read_csv('../data/clean_all_encounters_conditions_patients.csv')
psych_encounters = pd.read_csv('../data/clean_psych_encounters_conditions_patients.csv')
# patient_psych_diagnoses = pd.read_csv('../data/psych_diagnoses_per_patient.csv')
# psych_depts_encounters = pd.read_csv('../data/clean_psych_dept_encounters_conditions_patients.csv')
# nonpsych_depts_encounters = pd.read_csv('../data/clean_nonpsych_dept_encounters_conditions_patients.csv')

/var/folders/xm/m4dy5bvs1fn7fb9wxjhsg4dc0000gn/T/ipykernel_7465/889036862.py:1: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  all_encounters = pd.read_csv('../data/clean_all_encounters_conditions_patients.csv')


In [39]:
# psych_depts = pd.read_csv('/Users/valena17/DataGripProjects/msdw2/psych_depts/care_site_name.csv')
# psych_depts = list(psych_depts.xtn_department_name)
# psych_depts = [item for item in psych_depts if not(pd.isna(item))]
# psych_depts_encounters = all_encounters.loc[all_encounters.xtn_department_name.isin(psych_depts)]
# nonpsych_depts_encounters = all_encounters[~all_encounters['xtn_department_name'].isin(psych_depts)]
# psych_depts_encounters.head(20)
# pd.DataFrame.to_csv(psych_depts_encounters, "/Users/valena17/psych_nlp/data/clean_psych_dept_encounters_conditions_patients.csv",index = False)
# pd.DataFrame.to_csv(nonpsych_depts_encounters, "/Users/valena17/psych_nlp/data/clean_nonpsych_dept_encounters_conditions_patients.csv",index = False)

In [19]:
missing_i = pd.read_csv('/Users/valena17/DataGripProjects/msdw2/output/missing_insurance.csv',
                        names=['xtn_patient_epic_mrn', 'etl_epic_encounter_key', 'insurance_payor_class'],
                        usecols=[0,1,41])
missing_i = missing_i.replace(['*Unspecified', 'Other', 'Charge Correction'], np.nan)
missing_i = missing_i[missing_i.xtn_patient_epic_mrn.isin(mrns)]
all_insurance = pd.read_csv('/Users/valena17/psych_nlp/data/insurance/all_insurance.csv',
                        names=['xtn_patient_epic_mrn', 'etl_epic_encounter_key', 'insurance_payor_class'],
                        usecols=[0,1,41])
all_insurance = pd.concat([all_insurance, missing_i])
all_insurance
# pd.DataFrame.to_csv(all_insurance, "/Users/valena17/psych_nlp/data/insurance/insurance_complete.csv",index = False)
everything = pd.merge(all_encounters, all_insurance, how="left", on=["xtn_patient_epic_mrn", "etl_epic_encounter_key"])
everything

/var/folders/xm/m4dy5bvs1fn7fb9wxjhsg4dc0000gn/T/ipykernel_43879/758889035.py:7: DtypeWarning: Columns (0,1,2,3,4,5,30,31,35,36,37,38,52,53,55,56,57,58,59,60,61,64,65,66,67,68,69) have mixed types. Specify dtype option on import or set low_memory=False.
  all_insurance = pd.read_csv('/Users/valena17/psych_nlp/data/insurance/all_insurance.csv')


In [44]:
# everything = everything.drop(columns = ['insurance_payor_class_x', 'insured_status_x', 'time_since_zip_update', 'location_record_update_datetime', 'xtn_inpatient_admission_datetime', 'xtn_discharge_datetime', 'merged_race_ethnicity_v1'])
pd.DataFrame.to_csv(everything, "/Users/valena17/psych_nlp/data/clean_all_encounters_conditions_patients.csv",index = False)
# /Users/valena17/psych_nlp/data/clean_all_encounters_conditions_patients.csv most recent version as of 11/3/22

In [3]:
# save encounters with non na values for later analysis and save encounters with a psych diagnosis
# psych_encounters = df.dropna(axis = 0, subset=['merged_gender', 'insured_status_x', 'billing_condition', 'xtn_birth_date',
#                               'xtn_race_ethnicity_source_concept_name', 'visit_start_datetime', 'xtn_visit_type_source_concept_code'])
# everything = pd.read_csv('../data/clean_all_encounters_conditions_patients.csv')
psych_encounters = everything.groupby('visit_occurrence_id').filter(lambda x: x['psych_condition'].mean() > 0)
pd.DataFrame.to_csv(psych_encounters, "/Users/valena17/psych_nlp/data/clean_psych_encounters_conditions_patients.csv",index = False)

In [4]:
import pandas as pd
map = pd.read_csv("/Users/valena17/DataGripProjects/msdw2/psych_diagnoses/snomed_icd_map_v2.csv")
map = map[['referencedComponentName', 'mapTarget', 'mapTargetName']]
mapping_d = map.set_index('referencedComponentName').T.to_dict('list')
mapping_d["Dementia"]

/var/folders/xm/m4dy5bvs1fn7fb9wxjhsg4dc0000gn/T/ipykernel_64502/4174801071.py:4: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  mapping_d = map.set_index('referencedComponentName').T.to_dict('list')


['F03.90',
 'Unspecified dementia, unspecified severity, without behavioral disturbance, psychotic disturbance, mood disturbance, and anxiety']

In [5]:
def ICD_name(row):
    if row in mapping_d.keys():
        return mapping_d[row][1]

def ICD_code(row):
    if row in mapping_d.keys():
        return mapping_d[row][0]

all_encounters['ICD_name'] = all_encounters['condition_concept_name'].apply(ICD_name)
all_encounters['ICD_code'] = all_encounters['condition_concept_name'].apply(ICD_code)
all_encounters.head()

,person_id,visit_occurrence_id,etl_epic_encounter_key,xtn_department_name,xtn_visit_type_source_concept_id,xtn_visit_type_source_concept_code,visit_start_datetime,visit_end_datetime,xtn_patient_epic_mrn,xtn_birth_date,...,merged_gender,merged_race_ethnicity_v2,psych_condition,insurance_payor_class,insurance_status,ICD_name,ICD_code,ICD_code_h,ICD_code_top_h,CCSR_code
0,845,172058493,285636709,10 UNION SQ E RESP,2000000112,Outpatient Visit,2022-01-25 00:00:00,2022-01-25 00:00:00,3754282,1974-09-28,...,Female,White,0,Medicaid Managed Care,Public,None,None,NaN,NaN,NaN
1,845,172058493,285636709,10 UNION SQ E RESP,2000000112,Outpatient Visit,2022-01-25 00:00:00,2022-01-25 00:00:00,3754282,1974-09-28,...,Female,White,0,Medicaid Managed Care,Public,None,None,NaN,NaN,NaN
2,845,172058493,285636709,10 UNION SQ E RESP,2000000112,Outpatient Visit,2022-01-25 00:00:00,2022-01-25 00:00:00,3754282,1974-09-28,...,Female,White,1,Medicaid Managed Care,Public,None,None,F78.A,F78,Neurodevelopmental disorders
3,845,185133868,291089587,10 UNION SQ PFT,2000000112,Outpatient Visit,2022-02-22 00:00:00,2022-02-22 00:00:00,3754282,1974-09-28,...,Female,White,0,Medicaid Managed Care,Public,None,None,NaN,NaN,NaN
4,845,185133868,291089587,10 UNION SQ PFT,2000000112,Outpatient Visit,2022-02-22 00:00:00,2022-02-22 00:00:00,3754282,1974-09-28,...,Female,White,0,Medicaid Managed Care,Public,None,None,NaN,NaN,NaN


In [6]:
all_encounters['ICD_name'] = all_encounters['ICD_name'].replace(['None'], np.nan)
all_encounters['ICD_code'] = all_encounters['ICD_code'].replace(['None'], np.nan)

h = pd.read_csv("/Users/valena17/DataGripProjects/msdw2/psych_diagnoses/icd_fcode_hierarchy.csv")
h = h[['mapTarget', 'hierarchy', 'top_h']].drop_duplicates()
mapping_h = h.set_index('mapTarget').T.to_dict('list')
# mapping_h['F01.50']

def ICD_h(row):
    if row in mapping_h.keys():
        return mapping_h[row][0]

def ICD_h_top(row):
    if row in mapping_h.keys():
        return mapping_h[row][1]

all_encounters['ICD_code_h'] = all_encounters['ICD_code'].apply(ICD_h)
all_encounters['ICD_code_top_h'] = all_encounters['ICD_code'].apply(ICD_h_top)
all_encounters['ICD_code_h'] = all_encounters['ICD_code_h'].replace(['None'], np.nan)
all_encounters['ICD_code_top_h'] = all_encounters['ICD_code_top_h'].replace(['None'], np.nan)
all_encounters.head()

,person_id,visit_occurrence_id,etl_epic_encounter_key,xtn_department_name,xtn_visit_type_source_concept_id,xtn_visit_type_source_concept_code,visit_start_datetime,visit_end_datetime,xtn_patient_epic_mrn,xtn_birth_date,...,merged_gender,merged_race_ethnicity_v2,psych_condition,insurance_payor_class,insurance_status,ICD_name,ICD_code,ICD_code_h,ICD_code_top_h,CCSR_code
0,845,172058493,285636709,10 UNION SQ E RESP,2000000112,Outpatient Visit,2022-01-25 00:00:00,2022-01-25 00:00:00,3754282,1974-09-28,...,Female,White,0,Medicaid Managed Care,Public,None,None,None,None,NaN
1,845,172058493,285636709,10 UNION SQ E RESP,2000000112,Outpatient Visit,2022-01-25 00:00:00,2022-01-25 00:00:00,3754282,1974-09-28,...,Female,White,0,Medicaid Managed Care,Public,None,None,None,None,NaN
2,845,172058493,285636709,10 UNION SQ E RESP,2000000112,Outpatient Visit,2022-01-25 00:00:00,2022-01-25 00:00:00,3754282,1974-09-28,...,Female,White,1,Medicaid Managed Care,Public,None,None,None,None,Neurodevelopmental disorders
3,845,185133868,291089587,10 UNION SQ PFT,2000000112,Outpatient Visit,2022-02-22 00:00:00,2022-02-22 00:00:00,3754282,1974-09-28,...,Female,White,0,Medicaid Managed Care,Public,None,None,None,None,NaN
4,845,185133868,291089587,10 UNION SQ PFT,2000000112,Outpatient Visit,2022-02-22 00:00:00,2022-02-22 00:00:00,3754282,1974-09-28,...,Female,White,0,Medicaid Managed Care,Public,None,None,None,None,NaN


In [7]:
ccsr = pd.read_csv("/Users/valena17/DataGripProjects/msdw2/psych_diagnoses/ccsr_fcodes.csv")
ccsr = ccsr.applymap(lambda x: x.replace("'", ""))
ccsr = ccsr[['ICD-10-CM CODE', 'Default CCSR CATEGORY DESCRIPTION OP']]
ccsr_mapping = ccsr.set_index('ICD-10-CM CODE').T.to_dict('list')
ccsr_mapping['F01518']

['Neurocognitive disorders']

In [8]:
def CCSR(row):
    if row in mapping_h.keys():
        row = row.replace('.', '')
        return ccsr_mapping[row][0]

all_encounters['CCSR_code'] = all_encounters['ICD_code'].apply(CCSR)
all_encounters.head(50)

,person_id,visit_occurrence_id,etl_epic_encounter_key,xtn_department_name,xtn_visit_type_source_concept_id,xtn_visit_type_source_concept_code,visit_start_datetime,visit_end_datetime,xtn_patient_epic_mrn,xtn_birth_date,...,merged_gender,merged_race_ethnicity_v2,psych_condition,insurance_payor_class,insurance_status,ICD_name,ICD_code,ICD_code_h,ICD_code_top_h,CCSR_code
0,845,172058493,285636709,10 UNION SQ E RESP,2000000112,Outpatient Visit,2022-01-25 00:00:00,2022-01-25 00:00:00,3754282,1974-09-28,...,Female,White,0,Medicaid Managed Care,Public,None,None,None,None,None
1,845,172058493,285636709,10 UNION SQ E RESP,2000000112,Outpatient Visit,2022-01-25 00:00:00,2022-01-25 00:00:00,3754282,1974-09-28,...,Female,White,0,Medicaid Managed Care,Public,None,None,None,None,None
2,845,172058493,285636709,10 UNION SQ E RESP,2000000112,Outpatient Visit,2022-01-25 00:00:00,2022-01-25 00:00:00,3754282,1974-09-28,...,Female,White,1,Medicaid Managed Care,Public,None,None,None,None,None
3,845,185133868,291089587,10 UNION SQ PFT,2000000112,Outpatient Visit,2022-02-22 00:00:00,2022-02-22 00:00:00,3754282,1974-09-28,...,Female,White,0,Medicaid Managed Care,Public,None,None,None,None,None
4,845,185133868,291089587,10 UNION SQ PFT,2000000112,Outpatient Visit,2022-02-22 00:00:00,2022-02-22 00:00:00,3754282,1974-09-28,...,Female,White,0,Medicaid Managed Care,Public,None,None,None,None,None
5,845,185133868,291089587,10 UNION SQ PFT,2000000112,Outpatient Visit,2022-02-22 00:00:00,2022-02-22 00:00:00,3754282,1974-09-28,...,Female,White,1,Medicaid Managed Care,Public,None,None,None,None,None
6,845,194027752,293787212,17 E 102ND RHEUMATOLOGY CLINIC,2000000112,Outpatient Visit,2022-06-16 00:00:00,2022-06-16 00:00:00,3754282,1974-09-28,...,Female,White,0,Medicaid Managed Care,Public,None,None,None,None,None
7,845,194027752,293787212,17 E 102ND RHEUMATOLOGY CLINIC,2000000112,Outpatient Visit,2022-06-16 00:00:00,2022-06-16 00:00:00,3754282,1974-09-28,...,Female,White,0,Medicaid Managed Care,Public,None,None,None,None,None
8,845,194027752,293787212,17 E 102ND RHEUMATOLOGY CLINIC,2000000112,Outpatient Visit,2022-06-16 00:00:00,2022-06-16 00:00:00,3754282,1974-09-28,...,Female,White,0,Medicaid Managed Care,Public,None,None,None,None,None
9,845,197004100,302037736,17 E 102ND RHEUMATOLOGY CLINIC,2000000112,Outpatient Visit,2022-07-05 00:00:00,2022-07-05 00:00:00,3754282,1974-09-28,...,Female,White,0,Medicaid Managed Care,Public,None,None,None,None,None


In [9]:
map = pd.read_csv("/Users/valena17/DataGripProjects/msdw2/psych_diagnoses/snomed_icd_map_v2.csv")
map = map[['referencedComponentName', 'mapTarget', 'mapTargetName']]
mapping_d = map.set_index('referencedComponentName').T.to_dict('list')
mapping_d['Dementia']

diagnoses = set(map.referencedComponentName)
def condition_type(row):
    if row in diagnoses:
        return 1
    else:
        return 0

all_encounters['psych_condition'] = all_encounters.condition_concept_name.apply(lambda row: condition_type(row))


# all_encounters = pd.read_csv('../data/clean_all_encounters_conditions_patients.csv')
# psych_encounters = pd.read_csv('../data/clean_psych_encounters_conditions_patients.csv')

/var/folders/xm/m4dy5bvs1fn7fb9wxjhsg4dc0000gn/T/ipykernel_64502/3764839172.py:3: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  mapping_d = map.set_index('referencedComponentName').T.to_dict('list')


In [11]:
pd.DataFrame.to_csv(all_encounters, "/Users/valena17/psych_nlp/data/clean_all_encounters_conditions_patients.csv",index = False)

psych_encounters = all_encounters.groupby('visit_occurrence_id').filter(lambda x: x['psych_condition'].mean() > 0)
pd.DataFrame.to_csv(psych_encounters, "/Users/valena17/psych_nlp/data/clean_psych_encounters_conditions_patients.csv",index = False)

In [6]:
test = psych_encounters[(psych_encounters['psych_condition'] == 1)]
test = test.sort_values(by = ['person_id', 'visit_start_datetime'], ascending=True)
test = test.drop_duplicates(subset=['person_id','condition_concept_name'], keep='first')
test.head(50)

,person_id,visit_occurrence_id,etl_epic_encounter_key,xtn_department_name,xtn_visit_type_source_concept_id,xtn_visit_type_source_concept_code,visit_start_datetime,visit_end_datetime,xtn_patient_epic_mrn,xtn_birth_date,...,insurance_payor_class,insurance_status,ICD_name,ICD_code,ICD_code_h,ICD_code_top_h,CCSR_code,visit_year,birth_year,age_at_visit
12,913,198204422,304807149,341 CENTRAL PARK AVE INTERNAL MEDICINE,2000000112,Outpatient Visit,2022-07-21 00:00:00,2022-07-21 00:00:00,3657926,1952-03-02,...,Medicare MS,Public,"Anxiety disorder, unspecified",F41.9,F41.9,F41,Anxiety and fear-related disorders,2022,1952,70
19,913,198204422,304807149,341 CENTRAL PARK AVE INTERNAL MEDICINE,2000000112,Outpatient Visit,2022-07-21 00:00:00,2022-07-21 00:00:00,3657926,1952-03-02,...,Medicare MS,Public,"Depression, unspecified",F32.A,F32.A,F32,Depressive disorders,2022,1952,70
47,1081,195258082,297028242,37-12 82ND ST INTERNAL MEDICINE,2000000112,Outpatient Visit,2022-04-22 00:00:00,2022-04-22 00:00:00,3872637,1996-05-02,...,Medicaid Managed Care,Public,"Anxiety disorder, unspecified",F41.9,F41.9,F41,Anxiety and fear-related disorders,2022,1996,26
54,1256,195615221,297693162,1428 MADISON GENETICS,2000000112,Outpatient Visit,2022-06-17 00:00:00,2022-06-17 00:00:00,2460594,2002-07-28,...,Commercial/Managed Care,Private,Unspecified intellectual disabilities,F79,F79,F79,Neurodevelopmental disorders,2022,2002,20
63,1444,4162678,8321369,X_HOSP SBH MCSM MEDICAL_DEACTIVATED,2000000112,Outpatient Visit,2010-03-03 00:00:00,2010-03-03 00:00:00,2811095,1993-10-02,...,Self-Pay,Self-pay,"Anxiety disorder, unspecified",F41.9,F41.9,F41,Anxiety and fear-related disorders,2010,1993,17
66,1474,12354025,5567263,X_FPA GASTROENTEROLOGY_DEACTIVATED,2000000112,Outpatient Visit,2010-02-03,2010-02-03,3063678,1984-07-23,...,Self-Pay,Self-pay,"Bipolar disorder, unspecified",F31.9,F31.9,F31,Bipolar and related disorders,2010,1984,26
85,1564,11493815,6112675,X_HOSP PSYCH CHILD_DEACTIVATED,2000000112,Outpatient Visit,2010-04-02,2010-04-02,2177437,1994-02-11,...,Self-Pay,Self-pay,"Obsessive-compulsive disorder, unspecified",F42.9,F42.9,F42,Obsessive-compulsive and related disorders,2010,1994,16
76,1564,11493794,6064169,X_HOSP PSYCH CHILD_DEACTIVATED,2000000112,Outpatient Visit,2010-05-14,2010-05-14,2177437,1994-02-11,...,Self-Pay,Self-pay,"Attention-deficit hyperactivity disorder, unsp...",F90.9,F90.9,F90,Neurodevelopmental disorders,2010,1994,16
81,1564,11493813,6111967,X_HOSP PSYCH CHILD_DEACTIVATED,2000000112,Outpatient Visit,2010-06-18,2010-06-18,2177437,1994-02-11,...,Self-Pay,Self-pay,"Attention-deficit hyperactivity disorder, pred...",F90.0,F90.0,F90,Neurodevelopmental disorders,2010,1994,16
79,1564,11493804,9585911,X_HOSP PSYCH CHILD_DEACTIVATED,2000000112,Outpatient Visit,2010-08-30 00:00:00,2010-08-30 00:00:00,2177437,1994-02-11,...,Self-Pay,Self-pay,"Attention-deficit hyperactivity disorder, pred...",F90.1,F90.1,F90,Neurodevelopmental disorders,2010,1994,16


In [8]:
test.merged_race_ethnicity_v2.value_counts(dropna=False)
pd.DataFrame.to_csv(test, "/Users/valena17/psych_nlp/data/psych_diagnoses_per_patient.csv",index = False)

In [25]:
psych_encounters = psych_encounters.merged_race_ethnicity_v2.replace(['AMERICAN INDIAN OR ALASKAN', 'Hispanic/Latino', 'Hispanic', 'AFRICAN-AMERICAN', 'OTHER', 'PATIENT DECLINED', 'WHITE', 'ASIAN INDIAN', 'CHINESE',
                   'KOREAN', 'CHINESE AMERICAN', 'VIETNAMESE', 'OTHER: NORTH AFRICAN', 'JAPANESE', 'NATIVE HAWAIIAN', 'OTHER PACIFIC ISLANDER', 'THAI',
                   'PAPUA NEW GUINEAN', 'GUINEAN', 'FILIPINO', 'FILIPINO AMERICAN', 'BANGLADESHI', 'UGANDAN', 'PAKISTANI', 'WEST INDIAN', 'TRINIDADIAN', 'JAMAICAN', 'HAITIAN', 'TAIWANESE', 'NIGERIAN', 'ETHIOPIAN', 'DOMINICA ISLANDER', 'GRENADIAN', 'MADAGASCAR', 'POLYNESIAN', 'SRI LANKAN', 'INDONESIAN', 'MALIAN', 'GHANAIAN', 'BARBADIAN', 'TAHITIAN', 'ST VINCENTIAN', 'TOKELAUAN'],
                  ['American Indian or Alaska Native', 'Hispanic or Latino', 'Hispanic or Latino', 'Black or African American', 'Other Race', 'Decline to Answer', 'White', 'Asian', 'Asian', 'Asian', 'Asian', 'Asian', 'Black or African American', 'Asian', 'Native Hawaiian or Other Pacific Islander', 'Native Hawaiian or Other Pacific Islander', 'Asian', 'Native Hawaiian or Other Pacific Islander', 'Native Hawaiian or Other Pacific Islander', 'Native Hawaiian or Other Pacific Islander','Native Hawaiian or Other Pacific Islander', 'Asian', 'Black or African American', 'Asian', 'Asian', 'Mixed racial group', 'Black or African American', 'Black or African American', 'Asian', 'Black or African American', 'Black or African American', 'Black or African American', 'Black or African American', 'Black or African American', 'Native Hawaiian or Other Pacific Islander', 'Asian', 'Asian', 'Black or African American', 'Black or African American', 'Black or African American', 'Native Hawaiian or Other Pacific Islander', 'Black or African American', 'Native Hawaiian or Other Pacific Islander'])

# print(all_encounters.merged_race_ethnicity_v2.value_counts(dropna=False),
#       all_encounters.merged_gender.value_counts(dropna=False))

In [26]:
# pd.DataFrame.to_csv(test, "/Users/valena17/psych_nlp/data/psych_diagnoses_per_patient.csv",index = False)
pd.DataFrame.to_csv(all_encounters, "/Users/valena17/psych_nlp/data/clean_all_encounters_conditions_patients.csv",index = False)
pd.DataFrame.to_csv(psych_encounters, "/Users/valena17/psych_nlp/data/clean_psych_encounters_conditions_patients.csv",index = False)

In [152]:
header_list=['person_id', 'visit_occurrence_id', 'etl_epic_encounter_key', 'xtn_department_name', 'xtn_visit_type_source_concept_id',
             'xtn_visit_type_source_concept_code', 'visit_start_datetime', 'visit_end_datetime', 'xtn_inpatient_admission_datetime',
             'xtn_discharge_datetime', 'xtn_patient_epic_mrn', 'xtn_birth_date', 'xtn_race_ethnicity_source_concept_code',
             'xtn_race_ethnicity_source_concept_name', 'race_source_concept_code', 'race_source_concept_name',
             'gender_source_concept_code', 'gender_source_concept_name', 'xtn_gender_identity_source_concept_code',
             'xtn_gender_identity_source_concept_name', 'xtn_sexual_orientation_source_concept_code', 'xtn_sexual_orientation_source_concept_name', 'xtn_condition_status_source_concept_code', 'condition_concept_id', 'condition_concept_code', 'condition_concept_name', 'condition_start_datetime', 'xtn_five_digit_zip', 'location_record_update_datetime']
data1 = pd.read_csv('/Users/valena17/DataGripProjects/msdw2/output/testing_everything_3.csv', parse_dates=[6,7,8,9,11,26,28], names=header_list)
data2 = pd.read_csv('/Users/valena17/DataGripProjects/msdw2/output/testing_everything_3.2.csv', parse_dates=[6,7,8,9,11,26,28], names=header_list)
data = pd.concat([data1, data2])
print(data.nunique(axis=0))
pd.DataFrame.to_csv(data, "../data/psych_patient_encounter_conditions.csv",index = False)
data = data[data.person_id != 0]
data = data[data.visit_occurrence_id != 0]
data = data[data.etl_epic_encounter_key != 0]

/var/folders/xm/m4dy5bvs1fn7fb9wxjhsg4dc0000gn/T/ipykernel_71040/3448286411.py:7: DtypeWarning: Columns (14,16,27) have mixed types. Specify dtype option on import or set low_memory=False.
  data1 = pd.read_csv('/Users/valena17/DataGripProjects/msdw2/output/testing_everything_3.csv', parse_dates=[6,7,8,9,11,26,28], names=header_list)
/var/folders/xm/m4dy5bvs1fn7fb9wxjhsg4dc0000gn/T/ipykernel_71040/3448286411.py:8: DtypeWarning: Columns (14,16,27) have mixed types. Specify dtype option on import or set low_memory=False.
  data2 = pd.read_csv('/Users/valena17/DataGripProjects/msdw2/output/testing_everything_3.2.csv', parse_dates=[6,7,8,9,11,26,28], names=header_list)


In [75]:
# header_list=['person_id', 'visit_occurrence_id', 'etl_epic_encounter_key', 'xtn_department_name', 'xtn_visit_type_source_concept_id',
#              'xtn_visit_type_source_concept_code', 'visit_start_datetime', 'visit_end_datetime', 'xtn_inpatient_admission_datetime',
#              'xtn_discharge_datetime', 'xtn_patient_epic_mrn', 'xtn_birth_date', 'xtn_race_ethnicity_source_concept_code',
#              'xtn_race_ethnicity_source_concept_name', 'race_source_concept_code', 'race_source_concept_name',
#              'gender_source_concept_code', 'gender_source_concept_name', 'xtn_gender_identity_source_concept_code',
#              'xtn_gender_identity_source_concept_name', 'xtn_sexual_orientation_source_concept_code', 'xtn_sexual_orientation_source_concept_name', 'xtn_condition_status_source_concept_code', 'condition_concept_id', 'condition_concept_code', 'condition_concept_name', 'condition_start_datetime', 'xtn_five_digit_zip', 'location_record_update_datetime']
# df = pd.read_csv('/Users/valena17/DataGripProjects/msdw2/output/testing_everything.csv', parse_dates=[6,7,8,9,11,26,28], names=header_list)

,person_id,visit_occurrence_id,etl_epic_encounter_key,xtn_department_name,xtn_visit_type_source_concept_id,xtn_visit_type_source_concept_code,visit_start_datetime,visit_end_datetime,xtn_inpatient_admission_datetime,xtn_discharge_datetime,...,xtn_gender_identity_source_concept_name,xtn_sexual_orientation_source_concept_code,xtn_sexual_orientation_source_concept_name,xtn_condition_status_source_concept_code,condition_concept_id,condition_concept_code,condition_concept_name,condition_start_datetime,xtn_five_digit_zip,location_record_update_datetime
0,2115779,131485284,195731968,300 CADMAN PLAZA PEDIATRICS,2000000112,Outpatient Visit,2021-03-16 00:00:00,2021-03-16 00:00:00,NaT,NaT,...,*Unspecified,*Unspecified,*Unspecified,Billing Diagnosis,4144111,266435005,Gastroesophageal reflux disease without esopha...,2021-03-16,11224.0,2021-11-25 01:38:19.293
1,14546672,65247911,81508619,5 E 98 ENDOCRINOLOGY,2000000112,Outpatient Visit,2018-08-06 00:00:00,2018-08-06 00:00:00,NaT,NaT,...,Male,2,Straight (not lesbian or gay),Problem List,4159131,370992007,Dyslipidemia,2018-08-06,10033.0,2022-07-08 04:34:24.153
2,12970775,165648724,275164678,234 E 85TH ST NYEE MSH OPTH,2000000112,Outpatient Visit,2022-04-28 00:00:00,2022-04-28 00:00:00,NaT,NaT,...,Female,2,Straight (not lesbian or gay),Billing Diagnosis,4102183,193531003,Borderline glaucoma,2022-04-28,7748.0,2021-11-30 01:42:41.123
3,14303053,162816877,265688449,2109 BROADWAY IM,2000000112,Outpatient Visit,2021-08-12 00:00:00,2021-08-12 00:00:00,NaT,NaT,...,Male,2,Straight (not lesbian or gay),Billing Diagnosis,4245252,396152005,Raised prostate specific antigen,2021-08-12,10023.0,2022-06-14 01:52:50.580
4,1573247,29022672,194570630,X_1000 10TH AVE NEUROSURGERY_DEACTIVATED,2000000101,Telehealth Visit,2021-02-05 00:00:00,2021-02-05 00:00:00,NaT,NaT,...,*Unspecified,*Unspecified,*Unspecified,Billing Diagnosis,381270,49049000,Parkinson's disease,2021-02-05,11415.0,2021-11-17 01:37:36.797
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1333064,14473810,89302250,59216055,1190 5TH CARDIOVASC INST,2000000112,Outpatient Visit,2017-09-05 00:00:00,2017-09-05 00:00:00,NaT,NaT,...,Female,*Unspecified,*Unspecified,Encounter Diagnosis,312437,267036007,Dyspnea,2017-09-05,10026.0,2022-09-24 02:05:37.360
1333065,7163411,66680110,91154969,17 E 102ND IMA FIRM B,2000000112,Outpatient Visit,2019-02-28 00:00:00,2019-02-28 00:00:00,NaT,NaT,...,*Unspecified,*Unspecified,*Unspecified,Encounter Diagnosis,197672,165232002,Urinary incontinence,2019-02-28,10039.0,2021-11-07 08:57:12.287
1333066,14210676,163114499,266102317,MSQ 4 EAST,2000000099,Inpatient Hospitalization from ED Visit,2021-07-22 14:30:00,2021-07-26 11:40:00,2021-07-22 21:26:00,2021-07-26 11:40:00,...,*Unspecified,*Unspecified,*Unspecified,Billing Diagnosis,438878,166603001,Liver function tests abnormal,2021-07-23,11105.0,2021-11-07 09:06:27.550
1333067,1869791,31590330,182426190,N09C,2000000103,Inpatient Hospitalization,2020-06-11 10:04:00,2020-06-29 11:55:00,2020-06-11 10:04:00,2020-06-29 11:55:00,...,Male,*Unspecified,*Unspecified,Billing Diagnosis,4155910,271808008,"Edema, generalized",2020-06-12,11217.0,2022-01-08 07:25:42.590


In [163]:
# removing rows with same zip but different location update time
dup_zip=['person_id', 'visit_occurrence_id', 'etl_epic_encounter_key', 'xtn_department_name', 'xtn_visit_type_source_concept_id',
             'xtn_visit_type_source_concept_code', 'visit_start_datetime', 'visit_end_datetime', 'xtn_inpatient_admission_datetime',
             'xtn_discharge_datetime', 'xtn_patient_epic_mrn', 'xtn_birth_date', 'xtn_race_ethnicity_source_concept_code',
             'xtn_race_ethnicity_source_concept_name', 'race_source_concept_code', 'race_source_concept_name',
             'gender_source_concept_code', 'gender_source_concept_name', 'xtn_gender_identity_source_concept_code',
             'xtn_gender_identity_source_concept_name', 'xtn_sexual_orientation_source_concept_code', 'xtn_sexual_orientation_source_concept_name', 'xtn_condition_status_source_concept_code', 'condition_concept_id', 'condition_concept_code', 'condition_concept_name', 'condition_start_datetime', 'xtn_five_digit_zip']
df1 = data.drop_duplicates(subset=dup_zip, keep='last')

,person_id,visit_occurrence_id,etl_epic_encounter_key,xtn_department_name,xtn_visit_type_source_concept_id,xtn_visit_type_source_concept_code,visit_start_datetime,visit_end_datetime,xtn_inpatient_admission_datetime,xtn_discharge_datetime,...,xtn_gender_identity_source_concept_name,xtn_sexual_orientation_source_concept_code,xtn_sexual_orientation_source_concept_name,xtn_condition_status_source_concept_code,condition_concept_id,condition_concept_code,condition_concept_name,condition_start_datetime,xtn_five_digit_zip,location_record_update_datetime
4,4794633,169997280,10280319,MICU,2000000099,Inpatient Hospitalization from ED Visit,2011-09-03 20:18:00,2011-09-12 22:40:00,2011-09-03 22:05:00,2011-09-12 22:40:00,...,*Unspecified,*Unspecified,*Unspecified,Hospital Problem,4112853,254837009,Malignant tumor of breast,2011-09-03,11373.0,2021-11-17 01:36:00.597
6,4006329,70652611,62519800,234 E 85TH ST PEDS GEN,2000000112,Outpatient Visit,2017-11-08 00:00:00,2017-11-08 00:00:00,NaT,NaT,...,*Unspecified,*Unspecified,*Unspecified,Billing Diagnosis,314754,56018004,Wheezing,2017-11-08,10027.0,2022-10-07 02:06:31.147
11,2835428,36506151,186704819,17 E 102ND IMA FIRM D,2000000112,Outpatient Visit,2020-09-28 00:00:00,2020-09-28 00:00:00,NaT,NaT,...,Male,2,Straight (not lesbian or gay),Encounter Diagnosis,432875,413533008,Anemia due to chronic blood loss,2020-09-28,10039.0,2022-07-01 01:56:14.927
13,1952023,165988141,275833015,1436 BROADWAY INTERNAL MEDICINE,2000000112,Outpatient Visit,2021-12-14 00:00:00,2021-12-14 00:00:00,NaT,NaT,...,*Unspecified,*Unspecified,*Unspecified,Encounter Diagnosis,436070,34713006,Vitamin D deficiency,2021-12-14,11096.0,2022-03-08 01:45:30.133
28,13045813,42226805,48599413,X_FPA PAIN MGMT/ANESTHESIOLOGY,2000000112,Outpatient Visit,2016-06-23 00:00:00,2016-06-23 00:00:00,NaT,NaT,...,Female,*Unspecified,*Unspecified,Problem List,72418,298382003,Scoliosis deformity of spine,2016-06-23,10572.0,2022-10-02 02:13:05.497
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51631917,15004618,195545050,297568577,638 COLUMBUS AVE INTERNAL MEDICINE,2000000112,Outpatient Visit,2022-08-29 00:00:00,2022-08-29 00:00:00,NaT,NaT,...,Female,2,Straight (not lesbian or gay),Billing Diagnosis,436070,34713006,Vitamin D deficiency,2022-08-29,10026,2022-05-03 01:47:01.870
51631918,2726307,37237189,14502255,NS INTERNAL MEDICINE-325,2000000112,Outpatient Visit,2014-09-12 00:00:00,2014-09-12 00:00:00,NaT,NaT,...,Male,2,Straight (not lesbian or gay),Encounter Diagnosis,4029305,13644009,Hypercholesterolemia,2014-09-12,11725,2022-03-03 06:01:45.033
51631919,7815747,164280709,270776184,10 UNION SQ E COVID 19 COE,2000000112,Outpatient Visit,2021-09-15 00:00:00,2021-09-15 00:00:00,NaT,NaT,...,*Unspecified,*Unspecified,*Unspecified,Encounter Diagnosis,432883,191611001,"Recurrent major depressive episodes, moderate",2021-09-15,10451,2021-11-17 01:36:00.597
51631920,4448349,113764586,54334321,X_MS BI INT MED 14TH,2000000112,Outpatient Visit,2017-02-03 00:00:00,2017-02-03 00:00:00,NaT,NaT,...,*Unspecified,*Unspecified,*Unspecified,Encounter Diagnosis,255573,13645005,Chronic obstructive lung disease,2017-02-03,10011,2021-11-17 01:36:00.597


In [164]:
# creating new column with billing vs non billing codes
# print(df1['xtn_condition_status_source_concept_code'].value_counts())
def billing(row):
    if row['xtn_condition_status_source_concept_code'] == 'Billing Diagnosis':
        return 'Billing Diagnosis'
    else:
        return 'Non-Billing Diagnosis'
df1['billing_condition'] = df1.apply(lambda row: billing(row), axis=1)
# print(df1['billing_condition'].value_counts())

/var/folders/xm/m4dy5bvs1fn7fb9wxjhsg4dc0000gn/T/ipykernel_71040/3608161958.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['billing_condition'] = df1.apply(lambda row: billing(row), axis=1)


In [165]:
# removing rows with duplicated condition names but different condition types (billing vs non billing)
# if condition had a billing label, then that's what stayed
df1 = df1.sort_values(by=['person_id', 'visit_occurrence_id', 'condition_concept_name','billing_condition'])
dup_con_bill=['person_id', 'visit_occurrence_id', 'etl_epic_encounter_key', 'xtn_department_name', 'xtn_visit_type_source_concept_id',
             'xtn_visit_type_source_concept_code', 'visit_start_datetime', 'visit_end_datetime', 'xtn_inpatient_admission_datetime',
             'xtn_discharge_datetime', 'xtn_patient_epic_mrn', 'xtn_birth_date', 'xtn_race_ethnicity_source_concept_code',
             'xtn_race_ethnicity_source_concept_name', 'race_source_concept_code', 'race_source_concept_name',
             'gender_source_concept_code', 'gender_source_concept_name', 'xtn_gender_identity_source_concept_code',
             'xtn_gender_identity_source_concept_name', 'xtn_sexual_orientation_source_concept_code', 'xtn_sexual_orientation_source_concept_name', 'condition_concept_id', 'condition_concept_code', 'condition_concept_name', 'xtn_five_digit_zip']
df2 = df1.drop_duplicates(subset=dup_con_bill, keep='first')
# df2

In [166]:
# drop duplicated condition names with different condition start datetime
dup_con=['person_id', 'visit_occurrence_id', 'etl_epic_encounter_key', 'xtn_department_name', 'xtn_visit_type_source_concept_id',
             'xtn_visit_type_source_concept_code', 'visit_start_datetime', 'visit_end_datetime', 'xtn_inpatient_admission_datetime',
             'xtn_discharge_datetime', 'xtn_patient_epic_mrn', 'xtn_birth_date', 'xtn_race_ethnicity_source_concept_code',
             'xtn_race_ethnicity_source_concept_name', 'race_source_concept_code', 'race_source_concept_name',
             'gender_source_concept_code', 'gender_source_concept_name', 'xtn_gender_identity_source_concept_code',
             'xtn_gender_identity_source_concept_name', 'xtn_sexual_orientation_source_concept_code', 'xtn_sexual_orientation_source_concept_name', 'condition_concept_id', 'condition_concept_code', 'condition_concept_name', 'xtn_five_digit_zip', 'location_record_update_datetime', 'billing_condition']
df3 = df2.drop_duplicates(subset=dup_con, keep='first')
# df3

,person_id,visit_occurrence_id,etl_epic_encounter_key,xtn_department_name,xtn_visit_type_source_concept_id,xtn_visit_type_source_concept_code,visit_start_datetime,visit_end_datetime,xtn_inpatient_admission_datetime,xtn_discharge_datetime,...,xtn_sexual_orientation_source_concept_code,xtn_sexual_orientation_source_concept_name,xtn_condition_status_source_concept_code,condition_concept_id,condition_concept_code,condition_concept_name,condition_start_datetime,xtn_five_digit_zip,location_record_update_datetime,billing_condition
34025380,0,166406671,270120309,37-12 82ND ST INTERNAL MEDICINE,2000000112,Outpatient Visit,2021-11-15 00:00:00,2021-11-15 00:00:00,NaT,NaT,...,No matching concept,No matching concept,Encounter Diagnosis,320128,59621000,Essential hypertension,2021-11-15,*Unsp,2021-09-15 17:55:51.000,Non-Billing Diagnosis
10798610,0,166406835,268387146,10 E 102 PULMONARY,2000000112,Outpatient Visit,2021-11-09 00:00:00,2021-11-09 00:00:00,NaT,NaT,...,No matching concept,No matching concept,Billing Diagnosis,440615,71677004,Effect of exposure to external cause,2021-11-09,*Unsp,2021-09-15 17:55:51.000,Billing Diagnosis
7606946,0,166406888,282949502,122 AMSTERDAM SCH,2000000112,Outpatient Visit,2021-11-17 00:00:00,2021-11-17 00:00:00,NaT,NaT,...,No matching concept,No matching concept,Primary Diagnosis,81151,44465007,Sprain of ankle,2021-11-17,*Unsp,2021-09-15 17:55:51.000,Non-Billing Diagnosis
2010169,0,166408303,270757536,17 E 102ND ENDOCRINE CLINIC,2000000112,Outpatient Visit,2021-11-18 00:00:00,2021-11-18 00:00:00,NaT,NaT,...,No matching concept,No matching concept,Encounter Diagnosis,45757474,1481000119100,Diabetes mellitus type 2 without retinopathy,2021-11-18,*Unsp,2021-09-15 17:55:51.000,Non-Billing Diagnosis
9854934,0,166408421,282727736,HOSP ORTHOPEDICS PEDS,2000000112,Outpatient Visit,2021-11-22 00:00:00,2021-11-22 00:00:00,NaT,NaT,...,No matching concept,No matching concept,Primary Diagnosis,81723,5662003,Contusion of hand,2021-11-22,*Unsp,2021-09-15 17:55:51.000,Non-Billing Diagnosis
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29209183,15906895,200636137,310738382,19 E 98TH ST PSYCH,2000000101,Telehealth Visit,2022-10-03 00:00:00,2022-10-03 00:00:00,NaT,NaT,...,*Unspecified,*Unspecified,Billing Diagnosis,443617,430909002,Conduct disorder,2022-10-03,10024.0,2022-10-04 02:08:59.403,Billing Diagnosis
29214262,15906895,200636137,310738382,19 E 98TH ST PSYCH,2000000112,Outpatient Visit,2022-10-03 00:00:00,2022-10-03 00:00:00,NaT,NaT,...,*Unspecified,*Unspecified,Billing Diagnosis,443617,430909002,Conduct disorder,2022-10-03,10024.0,2022-10-04 02:08:59.403,Billing Diagnosis
4416258,15909331,63329267,88353760,X_EMERGENCY DEPT BI BROOKLYN_DEACTIVATED,2000000113,ED Visit,2018-12-19 14:21:00,2018-12-19 16:12:00,NaT,2018-12-19 16:12:00,...,*Unspecified,*Unspecified,Encounter Diagnosis,4113821,198288003,Anxiety state,2018-12-19,11236.0,2021-11-07 08:57:05.270,Non-Billing Diagnosis
4411115,15909331,63329267,88353760,X_EMERGENCY DEPT BI BROOKLYN_DEACTIVATED,2000000113,ED Visit,2018-12-19 14:21:00,2018-12-19 16:12:00,NaT,2018-12-19 16:12:00,...,*Unspecified,*Unspecified,Primary Diagnosis,4085100,281794004,Viral upper respiratory tract infection,2018-12-19,11236.0,2021-11-07 08:57:05.270,Non-Billing Diagnosis


In [168]:
df3['visit_start_datetime'] = pd.to_datetime(df3['visit_start_datetime'])
df3['visit_end_datetime'] = pd.to_datetime(df3['visit_end_datetime'])
df3['xtn_inpatient_admission_datetime'] = pd.to_datetime(df3['xtn_inpatient_admission_datetime'])
df3['xtn_discharge_datetime'] = pd.to_datetime(df3['xtn_discharge_datetime'])
df3['xtn_birth_date'] = pd.to_datetime(df3['xtn_birth_date'])
df3['condition_start_datetime'] = pd.to_datetime(df3['condition_start_datetime'])
df3['location_record_update_datetime'] = pd.to_datetime(df3['location_record_update_datetime'])

# print(df3['visit_start_datetime'].isna().sum())
u = df3.select_dtypes(include=['datetime'])
df3['visit_start_datetime'] = u.visit_start_datetime.fillna(u.xtn_inpatient_admission_datetime)
df3['visit_end_datetime'] = u.visit_start_datetime.fillna(u.xtn_discharge_datetime)
df3 = df3.dropna(subset=['visit_start_datetime'])
df3['time_since_zip_update'] = (df3.location_record_update_datetime - df3.visit_start_datetime)
# df3

In [170]:
dup_con=['person_id', 'visit_occurrence_id', 'etl_epic_encounter_key', 'xtn_department_name', 'xtn_visit_type_source_concept_id',
             'xtn_visit_type_source_concept_code', 'visit_start_datetime', 'visit_end_datetime', 'xtn_inpatient_admission_datetime',
             'xtn_discharge_datetime', 'xtn_patient_epic_mrn', 'xtn_birth_date', 'xtn_race_ethnicity_source_concept_code',
             'xtn_race_ethnicity_source_concept_name', 'race_source_concept_code', 'race_source_concept_name',
             'gender_source_concept_code', 'gender_source_concept_name', 'xtn_gender_identity_source_concept_code',
             'xtn_gender_identity_source_concept_name', 'xtn_sexual_orientation_source_concept_code', 'xtn_sexual_orientation_source_concept_name', 'condition_concept_id', 'condition_concept_code', 'condition_concept_name', 'billing_condition']
df4 = df3.sort_values(by=['time_since_zip_update', 'visit_occurrence_id'],ascending=[True, True], key=lambda ele: abs(ele))
df4 = df4.drop_duplicates(subset=dup_con,keep='first')
# df4

,person_id,visit_occurrence_id,etl_epic_encounter_key,xtn_department_name,xtn_visit_type_source_concept_id,xtn_visit_type_source_concept_code,visit_start_datetime,visit_end_datetime,xtn_inpatient_admission_datetime,xtn_discharge_datetime,...,xtn_sexual_orientation_source_concept_name,xtn_condition_status_source_concept_code,condition_concept_id,condition_concept_code,condition_concept_name,condition_start_datetime,xtn_five_digit_zip,location_record_update_datetime,billing_condition,time_since_zip_update
1466309,4457878,185041362,290911283,EMERGENCY DEPT BI,2000000113,ED Visit,2022-02-21 01:46:00,2022-02-21 01:46:00,NaT,2022-02-21 04:12:00,...,*Unspecified,Encounter Diagnosis,25297,363746003,Acute pharyngitis,2022-02-21,11237.0,2022-02-21 01:47:57.067,Non-Billing Diagnosis,0 days 00:01:57.067000
1469639,4457878,185041362,290911283,EMERGENCY DEPT BI,2000000113,ED Visit,2022-02-21 01:46:00,2022-02-21 01:46:00,NaT,2022-02-21 04:12:00,...,*Unspecified,Primary Diagnosis,442077,197480006,Anxiety disorder,2022-02-21,11237.0,2022-02-21 01:47:57.067,Non-Billing Diagnosis,0 days 00:01:57.067000
7080027,1634822,193781739,293267531,BABCOCK 4W,2000000099,Inpatient Hospitalization from ED Visit,2022-03-10 01:33:00,2022-03-10 01:33:00,2022-03-10 01:43:00,2022-03-18 16:46:00,...,*Unspecified,Billing Diagnosis,37016348,367991000119101,Hyperglycemia due to type 1 diabetes mellitus,2022-03-11,10025.0,2022-03-10 01:45:51.287,Billing Diagnosis,0 days 00:12:51.287000
7085103,1634822,193781739,293267531,BABCOCK 4W,2000000099,Inpatient Hospitalization from ED Visit,2022-03-10 01:33:00,2022-03-10 01:33:00,2022-03-10 01:43:00,2022-03-18 16:46:00,...,*Unspecified,Billing Diagnosis,443412,313435000,Type 1 diabetes mellitus without complication,2022-03-16,10025.0,2022-03-10 01:45:51.287,Billing Diagnosis,0 days 00:12:51.287000
7082361,1634822,193781739,293267531,BABCOCK 4W,2000000099,Inpatient Hospitalization from ED Visit,2022-03-10 01:33:00,2022-03-10 01:33:00,2022-03-10 01:43:00,2022-03-18 16:46:00,...,*Unspecified,Problem List,201826,44054006,Type 2 diabetes mellitus,2022-03-10,10025.0,2022-03-10 01:45:51.287,Non-Billing Diagnosis,0 days 00:12:51.287000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28612207,5051945,3017558,5374347,X_GERIATRICS,2000000112,Outpatient Visit,2006-06-29 00:00:00,2006-06-29 00:00:00,NaT,NaT,...,*Unspecified,Encounter Diagnosis,201820,73211009,Diabetes mellitus,2006-06-29,11372,2022-10-02 02:13:05.497,Non-Billing Diagnosis,5939 days 02:13:05.497000
28614278,5051945,3017558,5374347,X_GERIATRICS,2000000112,Outpatient Visit,2006-06-29 00:00:00,2006-06-29 00:00:00,NaT,NaT,...,*Unspecified,Encounter Diagnosis,316866,38341003,Hypertensive disorder,2006-06-29,11372,2022-10-02 02:13:05.497,Non-Billing Diagnosis,5939 days 02:13:05.497000
28610930,5051945,3017558,5374347,X_GERIATRICS,2000000112,Outpatient Visit,2006-06-29 00:00:00,2006-06-29 00:00:00,NaT,NaT,...,*Unspecified,Encounter Diagnosis,374375,18070006,Impacted cerumen,2006-06-29,11372,2022-10-02 02:13:05.497,Non-Billing Diagnosis,5939 days 02:13:05.497000
28608592,5051945,3017558,5374347,X_GERIATRICS,2000000112,Outpatient Visit,2006-06-29 00:00:00,2006-06-29 00:00:00,NaT,NaT,...,*Unspecified,Encounter Diagnosis,192450,267064002,Retention of urine,2006-06-29,11372,2022-10-02 02:13:05.497,Non-Billing Diagnosis,5939 days 02:13:05.497000


In [1]:
# df = pd.read_csv('../data/clean_psych_patient_encounter_conditions.csv')
# print(df.head(10))
# df = df.replace(['UNKNOWN','Race', 'No matching concept', 'Unknown', '*Unspecified', '*Unknown'], np.nan)
# df['merged_gender'] = df.xtn_gender_identity_source_concept_name.fillna(df.gender_source_concept_name)
# df['merged_race_ethnicity_v1'] = df.race_source_concept_name.fillna(df.xtn_race_ethnicity_source_concept_name)
# df['merged_race_ethnicity_v2'] = df.xtn_race_ethnicity_source_concept_name.fillna(df.race_source_concept_name)
#
# insurance = pd.read_csv("../data/insurance/all_insurance.csv",
#                         names=['xtn_patient_epic_mrn', 'etl_epic_encounter_key', 'insurance_payor_class'],
#                         usecols=[0,1,41])
# insurance = insurance.replace(['*Unspecified', 'Other', 'Charge Correction'], np.nan)

# def insurance_type(row):
#     if search('Self', str(row)):
#         return 'Uninsured'
#     else:
#         return 'Insured'
#
# insurance['insured_status'] = insurance.insurance_payor_class.apply(lambda row: insurance_type(row))
# print(insurance.head(20))
# insurance_data = pd.merge(df, insurance, how="left", on=["xtn_patient_epic_mrn", "etl_epic_encounter_key"])
# print(insurance_data.head(20))
# print(insurance.nunique(axis=0))
# pd.DataFrame.to_csv(insurance_data, "../data/clean_psych_patient_encounter_conditions.csv",index = False)

NameError: name 'pd' is not defined

In [3]:
psych_encounters = pd.read_csv('../data/clean_psych_encounters_conditions_patients.csv')
psych_encounters

/var/folders/xm/m4dy5bvs1fn7fb9wxjhsg4dc0000gn/T/ipykernel_29106/2864821143.py:1: DtypeWarning: Columns (10,21) have mixed types. Specify dtype option on import or set low_memory=False.
  psych_encounters = pd.read_csv('../data/clean_psych_patient_encounter_conditions.csv')


,person_id,visit_occurrence_id,etl_epic_encounter_key,xtn_department_name,xtn_visit_type_source_concept_id,xtn_visit_type_source_concept_code,visit_start_datetime,visit_end_datetime,xtn_inpatient_admission_datetime,xtn_discharge_datetime,...,xtn_five_digit_zip,location_record_update_datetime,billing_condition,time_since_zip_update,merged_gender,insurance_payor_class_x,insured_status_x,merged_race_ethnicity_v1,merged_race_ethnicity_v2,psych_condition
0,845,172058493,285636709,10 UNION SQ E RESP,2000000112,Outpatient Visit,2022-01-25 00:00:00,2022-01-25 00:00:00,NaN,NaN,...,11234.0,2022-08-25 01:53:08.583,Billing Diagnosis,212 days 01:53:08.583000,Female,Medicaid Managed Care,Insured,WHITE,White,0
1,845,172058493,285636709,10 UNION SQ E RESP,2000000112,Outpatient Visit,2022-01-25 00:00:00,2022-01-25 00:00:00,NaN,NaN,...,11234.0,2022-08-25 01:53:08.583,Billing Diagnosis,212 days 01:53:08.583000,Female,Medicaid Managed Care,Insured,WHITE,White,0
2,845,172058493,285636709,10 UNION SQ E RESP,2000000112,Outpatient Visit,2022-01-25 00:00:00,2022-01-25 00:00:00,NaN,NaN,...,11234.0,2022-08-25 01:53:08.583,Non-Billing Diagnosis,212 days 01:53:08.583000,Female,Medicaid Managed Care,Insured,WHITE,White,1
3,845,185133868,291089587,10 UNION SQ PFT,2000000112,Outpatient Visit,2022-02-22 00:00:00,2022-02-22 00:00:00,NaN,NaN,...,11234.0,2022-08-25 01:53:08.583,Billing Diagnosis,184 days 01:53:08.583000,Female,NaN,NaN,WHITE,White,0
4,845,185133868,291089587,10 UNION SQ PFT,2000000112,Outpatient Visit,2022-02-22 00:00:00,2022-02-22 00:00:00,NaN,NaN,...,11234.0,2022-08-25 01:53:08.583,Billing Diagnosis,184 days 01:53:08.583000,Female,NaN,NaN,WHITE,White,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28321985,15906895,200636137,310738382,19 E 98TH ST PSYCH,2000000101,Telehealth Visit,2022-10-03 00:00:00,2022-10-03 00:00:00,NaN,NaN,...,10024.0,2022-10-04 02:08:59.403,Billing Diagnosis,1 days 02:08:59.403000,Female,Self-Pay,Uninsured,WHITE,White,1
28321986,15906895,200636137,310738382,19 E 98TH ST PSYCH,2000000112,Outpatient Visit,2022-10-03 00:00:00,2022-10-03 00:00:00,NaN,NaN,...,10024.0,2022-10-04 02:08:59.403,Billing Diagnosis,1 days 02:08:59.403000,Female,Self-Pay,Uninsured,WHITE,White,1
28321987,15909331,63329267,88353760,X_EMERGENCY DEPT BI BROOKLYN_DEACTIVATED,2000000113,ED Visit,2018-12-19 14:21:00,2018-12-19 14:21:00,NaN,2018-12-19 16:12:00,...,11236.0,2021-11-07 08:57:05.270,Non-Billing Diagnosis,1053 days 18:36:05.270000,Female,NaN,NaN,NaN,NaN,1
28321988,15909331,63329267,88353760,X_EMERGENCY DEPT BI BROOKLYN_DEACTIVATED,2000000113,ED Visit,2018-12-19 14:21:00,2018-12-19 14:21:00,NaN,2018-12-19 16:12:00,...,11236.0,2021-11-07 08:57:05.270,Non-Billing Diagnosis,1053 days 18:36:05.270000,Female,NaN,NaN,NaN,NaN,0


In [3]:
checking_gender = psych_encounters.dropna(axis = 0, subset=['gender_source_concept_name', 'xtn_gender_identity_source_concept_name'])
checking_gender = checking_gender.loc[checking_gender['gender_source_concept_name'] != checking_gender['xtn_gender_identity_source_concept_name']]
checking_gender = checking_gender.drop_duplicates(subset=['person_id', 'gender_source_concept_name', 'xtn_gender_identity_source_concept_name', 'merged_gender'])
checking_gender = checking_gender[['person_id', 'gender_source_concept_name', 'xtn_gender_identity_source_concept_name', 'merged_gender', 'xtn_sexual_orientation_source_concept_name']]
checking_gender

,person_id,gender_source_concept_name,xtn_gender_identity_source_concept_name,merged_gender,xtn_sexual_orientation_source_concept_name
195,13851,Male,Transgender Female / Male-to-Female,Transgender Female / Male-to-Female,NaN
513,39637,Female,Male,Male,NaN
906,95223,Female,Male,Male,NaN
992,113216,Male,Non-binary,Non-binary,NaN
1287,168106,Female,Male,Male,Queer
...,...,...,...,...,...
8481614,15889420,Male,Female,Female,Straight (not lesbian or gay)
8481741,15890391,Female,Transgender Male / Female-to-Male,Transgender Male / Female-to-Male,NaN
8481807,15890780,Female,Non-binary,Non-binary,NaN
8481949,15891821,Female,Transgender Male / Female-to-Male,Transgender Male / Female-to-Male,Queer


In [38]:
# import math
# renaming insurance payor values
def insurance_names(row): # WTC,
    if search('dicare|dicaid|WTC', str(row)):
        return 'Public'
    if search('Self', str(row)):
        return 'Self-pay'
    if pd.isna(row):
        return 'NaN'
    else:
        return 'Private'

everything['insurance_status'] = everything['insurance_payor_class'].apply(insurance_names)
everything

,person_id,visit_occurrence_id,etl_epic_encounter_key,xtn_department_name,xtn_visit_type_source_concept_id,xtn_visit_type_source_concept_code,visit_start_datetime,visit_end_datetime,xtn_patient_epic_mrn,xtn_birth_date,...,condition_concept_code,condition_concept_name,condition_start_datetime,xtn_five_digit_zip,billing_condition,merged_gender,merged_race_ethnicity_v2,psych_condition,insurance_payor_class,insurance_status
0,845,172058493,285636709,10 UNION SQ E RESP,2000000112,Outpatient Visit,2022-01-25 00:00:00,2022-01-25 00:00:00,3754282,1974-09-28,...,193462001,Insomnia,2022-01-25,11234.0,Billing Diagnosis,Female,White,0,Medicaid Managed Care,Public
1,845,172058493,285636709,10 UNION SQ E RESP,2000000112,Outpatient Visit,2022-01-25 00:00:00,2022-01-25 00:00:00,3754282,1974-09-28,...,1101000119103,Obstructive sleep apnea of adult,2022-01-25,11234.0,Billing Diagnosis,Female,White,0,Medicaid Managed Care,Public
2,845,172058493,285636709,10 UNION SQ E RESP,2000000112,Outpatient Visit,2022-01-25 00:00:00,2022-01-25 00:00:00,3754282,1974-09-28,...,78275009,Obstructive sleep apnea syndrome,2022-01-25,11234.0,Non-Billing Diagnosis,Female,White,1,Medicaid Managed Care,Public
3,845,185133868,291089587,10 UNION SQ PFT,2000000112,Outpatient Visit,2022-02-22 00:00:00,2022-02-22 00:00:00,3754282,1974-09-28,...,193462001,Insomnia,2022-02-22,11234.0,Billing Diagnosis,Female,White,0,Medicaid Managed Care,Public
4,845,185133868,291089587,10 UNION SQ PFT,2000000112,Outpatient Visit,2022-02-22 00:00:00,2022-02-22 00:00:00,3754282,1974-09-28,...,1101000119103,Obstructive sleep apnea of adult,2022-02-22,11234.0,Billing Diagnosis,Female,White,0,Medicaid Managed Care,Public
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28326236,15906895,200636137,310738382,19 E 98TH ST PSYCH,2000000101,Telehealth Visit,2022-10-03 00:00:00,2022-10-03 00:00:00,F801514,2013-04-15,...,430909002,Conduct disorder,2022-10-03,10024.0,Billing Diagnosis,Female,White,1,Self-Pay,Self-pay
28326237,15906895,200636137,310738382,19 E 98TH ST PSYCH,2000000112,Outpatient Visit,2022-10-03 00:00:00,2022-10-03 00:00:00,F801514,2013-04-15,...,430909002,Conduct disorder,2022-10-03,10024.0,Billing Diagnosis,Female,White,1,Self-Pay,Self-pay
28326238,15909331,63329267,88353760,X_EMERGENCY DEPT BI BROOKLYN_DEACTIVATED,2000000113,ED Visit,2018-12-19 14:21:00,2018-12-19 14:21:00,F805539,1989-07-24,...,198288003,Anxiety state,2018-12-19,11236.0,Non-Billing Diagnosis,Female,NaN,1,Commercial/Managed Care,Private
28326239,15909331,63329267,88353760,X_EMERGENCY DEPT BI BROOKLYN_DEACTIVATED,2000000113,ED Visit,2018-12-19 14:21:00,2018-12-19 14:21:00,F805539,1989-07-24,...,281794004,Viral upper respiratory tract infection,2018-12-19,11236.0,Non-Billing Diagnosis,Female,NaN,0,Commercial/Managed Care,Private


In [7]:
# psych_encounters.head(200)
# everything.replace('*Unspecified', 'NaN')
all_encounters.insurance_status.value_counts(dropna=False)

Self-pay    11031803
Public      10865554
Private      6216742
NaN           212142
Name: insurance_status, dtype: int64

In [10]:
missing_insurance = all_encounters[all_encounters['insurance_payor_class'].isna()]
missing_insurance = missing_insurance[['person_id','xtn_patient_epic_mrn', 'visit_occurrence_id', 'etl_epic_encounter_key']].drop_duplicates()
missing_insurance

,person_id,xtn_patient_epic_mrn,visit_occurrence_id,etl_epic_encounter_key
5805,417194,4093792,197232273,302254463
11932,501856,8155832,165735466,275308185
11936,501856,8155832,165811527,275400221
28116,612988,6836009,186287758,57523938
40687,617308,1158593,79082629,60122918
...,...,...,...,...
28324402,15890360,F787518,200063121,309771971
28325006,15893414,F790462,200171316,309956343
28325223,15895244,F792080,200236582,310051542
28325850,15901612,F795935,200518759,310299378


In [11]:
pd.DataFrame.to_csv(missing_insurance, "../data/missing_insurance.csv",index = True)

In [55]:
map = pd.read_csv("/Users/valena17/DataGripProjects/msdw2/psych_diagnoses/snomed_icd_map.csv")
map = map[['referencedComponentName', 'mapTarget', 'mapTargetName']]
mapping_d = map.set_index('referencedComponentName').T.to_dict('list')
mapping_d['Failure of orgasm']

/var/folders/xm/m4dy5bvs1fn7fb9wxjhsg4dc0000gn/T/ipykernel_6367/2861663421.py:3: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  mapping_d = map.set_index('referencedComponentName').T.to_dict('list')


['F52.32', 'Male orgasmic disorder']

In [85]:
import seaborn as sns
import matplotlib.pyplot as plt
g = psych_encounters.groupby(['billing_condition','ICD_hierarchy','insured_status_x'])
condition_cnt = g.person_id.nunique() #.sort_values(by=['time_since_zip_update', 'visit_occurrence_id'],ascending=[True, True]
condition_cnt

billing_condition      ICD_hierarchy  insured_status_x
Billing Diagnosis      F01.5          Insured             2357
                                      Uninsured             48
                       F02.8          Insured             5584
                                      Uninsured            128
                       F03.9          Insured             5762
                                                          ... 
Non-Billing Diagnosis  F98.5          Uninsured             38
                       F98.9          Insured               12
                                      Uninsured             45
                       F99            Insured              312
                                      Uninsured            575
Name: person_id, Length: 663, dtype: int64

In [45]:
# this makes condition matrix with more broad CCSR code hierarchy with no decimal place
CCSR = set(all_encounters.CCSR_code)

def count_conditions(group):
    total_c = group['condition_concept_name'].count()
    # total_p_c = group['ICD_code'].count()
    c_counts = list()
    c_counts.append(group.name)
    for code in CCSR:
        p_count = group[group.CCSR_code == code].shape[0] # group['ICD_code'].value_counts()[psych_code]
        c_counts.append(float(p_count/total_c))
    return c_counts

condition_count_matrix_CCSR= all_encounters.groupby('person_id').apply(lambda group: count_conditions(group))
pd.DataFrame.to_csv(condition_count_matrix_CCSR, "../data/all_encounters_condition_count_matrix_CCSR.csv",index = False)
condition_count_matrix_CCSR

person_id
845         [845, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
913         [913, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
1081        [1081, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...
1256        [1256, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...
1351        [1351, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...
                                  ...                        
15906752    [15906752, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
15906832    [15906832, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, ...
15906880    [15906880, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
15906895    [15906895, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
15909331    [15909331, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
Length: 466692, dtype: object

In [4]:
## this makes condition matrix with more specific ICD code hierarchy with one decimal place
ICD = set(all_encounters.ICD_code_h)

def count_conditions(group):
    total_c = group['condition_concept_name'].count()
    # total_p_c = group['ICD_code'].count()
    c_counts = list()
    c_counts.append(group.name)
    for code in ICD:
        p_count = group[group.ICD_code_h == code].shape[0] # group['ICD_code'].value_counts()[psych_code]
        c_counts.append(float(p_count/total_c))
    return c_counts

condition_count_matrix_ICD= all_encounters.groupby('person_id').apply(lambda group: count_conditions(group))
pd.DataFrame.to_csv(condition_count_matrix_ICD, "../data/all_encounters_condition_count_matrix_ICD.csv",index = False)
condition_count_matrix_ICD

person_id
845         [845, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
913         [913, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
1081        [1081, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...
1256        [1256, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...
1351        [1351, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...
                                  ...                        
15906752    [15906752, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
15906832    [15906832, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
15906880    [15906880, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
15906895    [15906895, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
15909331    [15909331, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
Length: 466692, dtype: object

In [9]:
condition_count_matrix_CCSR = pd.read_csv("../data/all_encounters_condition_count_matrix_CCSR.csv")
condition_count_matrix_ICD = pd.read_csv("../data/all_encounters_condition_count_matrix_ICD.csv")

In [48]:
# patient_data = insurance_data[['person_id', 'xtn_patient_epic_mrn', 'visit_start_datetime', 'xtn_birth_date', 'xtn_race_ethnicity_source_concept_code',
#                                'xtn_race_ethnicity_source_concept_name', 'race_source_concept_code', 'race_source_concept_name', 'merged_race_ethnicity_v1',
#                                'merged_race_ethnicity_v2', 'gender_source_concept_code', 'gender_source_concept_name', 'xtn_gender_identity_source_concept_code', 'merged_gender', 'xtn_gender_identity_source_concept_name', 'xtn_sexual_orientation_source_concept_code', 'xtn_sexual_orientation_source_concept_name', 'xtn_five_digit_zip']]
# p = pd.read_csv("../data/clean_psych_patient.csv")
# idk = list(p['person_id'])

# psych_codes = set(psych_encounters.ICD_top_h)
# test = list(psych_codes)
# test = ['person_id'] + test # test.insert(0, 'person_id')


CCSR = set(all_encounters.CCSR_code)
CCSR = list(CCSR)
CCSR = ['person_id'] + CCSR
everything_CCSR = []
for line in condition_count_matrix_CCSR:
    everything_CCSR.append(line)
CCSR_matrix = pd.DataFrame(data=everything_CCSR, columns=[CCSR])
CCSR_matrix = CCSR_matrix.sort_index(axis = 1)
CCSR_matrix = CCSR_matrix.iloc[: , :-1]
print(CCSR_matrix.head(10))

# ICD = set(all_encounters.ICD_code_h)
# ICD = list(ICD)
# ICD = ['person_id'] + ICD
# everything_ICD = []
# for group in condition_count_matrix_ICD:
#     everything_ICD.append(group)
# ICD_matrix = pd.DataFrame(everything_ICD, columns=[ICD])
# ICD_matrix = ICD_matrix.sort_index(axis = 1)
# ICD_matrix = ICD_matrix.iloc[: , :-1]
# print(ICD_matrix.head(10))
#
# p = pd.read_csv("../data/clean_psych_patient.csv")

  Alcohol-related disorders Anxiety and fear-related disorders  \
0                       0.0                           0.000000   
1                       0.0                           0.028571   
2                       0.0                           0.142857   
3                       0.0                           0.000000   
4                       0.0                           0.000000   
5                       0.0                           0.200000   
6                       0.0                           0.000000   
7                       0.0                           0.000000   
8                       0.0                           0.000000   
9                       0.0                           0.000000   

  Bipolar and related disorders Cannabis-related disorders  \
0                          0.00                        0.0   
1                          0.00                        0.0   
2                          0.00                        0.0   
3                        

In [49]:
# print(CCSR_matrix.columns.map(''.join))
p = pd.read_csv("../data/clean_psych_patient.csv")
CCSR_matrix.columns = CCSR_matrix.columns.map(''.join)
CCSR_matrix = pd.concat([CCSR_matrix, p], axis=1).reset_index()
CCSR_matrix = CCSR_matrix.drop(columns='index')
print(CCSR_matrix.head(10))
pd.DataFrame.to_csv(CCSR_matrix, "../data/all_encounters_condition_count_matrix_CCSR.csv",index = False)

# ICD_matrix.columns = ICD_matrix.columns.map(''.join)
# ICD_matrix = pd.concat([ICD_matrix, p], axis=1).reset_index()
# ICD_matrix = ICD_matrix.drop(columns='index')
# print(ICD_matrix.head(10))
# pd.DataFrame.to_csv(ICD_matrix, "../data/all_encounters_condition_count_matrix_ICD.csv",index = False)

   Alcohol-related disorders  Anxiety and fear-related disorders  \
0                        0.0                            0.000000   
1                        0.0                            0.028571   
2                        0.0                            0.142857   
3                        0.0                            0.000000   
4                        0.0                            0.000000   
5                        0.0                            0.200000   
6                        0.0                            0.000000   
7                        0.0                            0.000000   
8                        0.0                            0.000000   
9                        0.0                            0.000000   

   Bipolar and related disorders  Cannabis-related disorders  \
0                           0.00                         0.0   
1                           0.00                         0.0   
2                           0.00                         0.

In [50]:
# this makes condition matrix with more broad CCSR code hierarchy with no decimal place
CCSR = set(psych_encounters.CCSR_code)

def count_conditions(group):
    total_c = group['condition_concept_name'].count()
    # total_p_c = group['ICD_code'].count()
    c_counts = list()
    c_counts.append(group.name)
    for code in CCSR:
        p_count = group[group.CCSR_code == code].shape[0] # group['ICD_code'].value_counts()[psych_code]
        c_counts.append(float(p_count/total_c))
    return c_counts

psych_condition_count_matrix_CCSR = psych_encounters.groupby('person_id').apply(lambda group: count_conditions(group))
pd.DataFrame.to_csv(psych_condition_count_matrix_CCSR, "../data/psych_encounters_condition_count_matrix_CCSR.csv",index = False)
# psych_condition_count_matrix_CCSR

CCSR = list(CCSR)
CCSR = ['person_id'] + CCSR
everything_CCSR = []
for line in psych_condition_count_matrix_CCSR:
    everything_CCSR.append(line)
CCSR_matrix = pd.DataFrame(data=everything_CCSR, columns=[CCSR])
CCSR_matrix = CCSR_matrix.sort_index(axis = 1)
CCSR_matrix = CCSR_matrix.iloc[: , :-1]
# print(CCSR_matrix.head(10))

p = pd.read_csv("../data/clean_psych_patient.csv")
CCSR_matrix.columns = CCSR_matrix.columns.map(''.join)
CCSR_matrix = pd.concat([CCSR_matrix, p], axis=1).reset_index()
CCSR_matrix = CCSR_matrix.drop(columns='index')
# print(CCSR_matrix.head(10))
pd.DataFrame.to_csv(CCSR_matrix, "../data/psych_encounters_condition_count_matrix_CCSR.csv",index = False)

In [217]:
print(#psych_encounters.condition_concept_name.value_counts(dropna=False),
      psych_encounters.groupby(['person_id'])['insurance_payor_class_x'].nunique().median(),
    psych_encounters.groupby(['person_id'])['insurance_payor_class_x'].nunique().max(),
    psych_encounters.groupby(['person_id'])['insurance_payor_class_x'].nunique().min())

# print(
#     psych_encounters.groupby(['person_id'])['visit_occurrence_id'].nunique().median(),
#     psych_encounters.groupby(['person_id'])['visit_occurrence_id'].nunique().max(),
#     psych_encounters.groupby(['person_id'])['visit_occurrence_id'].nunique().min()
# )
# print(psych_encounters.person_id.value_counts().mean(),
# psych_encounters.person_id.value_counts().std(),
# psych_encounters.person_id.value_counts().median(),
# psych_encounters.person_id.value_counts().max(),
# psych_encounters.person_id.value_counts().min())
#
# print(psych_encounters.visit_occurrence_id.value_counts().mean(),
# psych_encounters.visit_occurrence_id.value_counts().std(),
# psych_encounters.visit_occurrence_id.value_counts().median(),
# psych_encounters.visit_occurrence_id.value_counts().max(),
# psych_encounters.visit_occurrence_id.value_counts().min())
#
# print(psych_encounters.xtn_department_name.value_counts(dropna=False),
# psych_encounters.xtn_visit_type_source_concept_code.value_counts(dropna=False))
#
# print(psych_encounters.visit_start_datetime.max(),
# psych_encounters.visit_start_datetime.min(),
# psych_encounters.visit_start_datetime.isna().sum(),
# psych_encounters.visit_end_datetime.max(),
# psych_encounters.visit_end_datetime.min(),
# psych_encounters.visit_end_datetime.isna().sum())
#
# print(psych_encounters.xtn_five_digit_zip.value_counts(dropna=False))
#
# print(psych_encounters.insurance_payor_class_x.value_counts(dropna=False),
#       psych_encounters.insured_status_x.value_counts(dropna=False))

1.0 8 0


In [ ]:
 df2 = pd.read_csv("../data/clean_psych_patient.csv")

print(df2.race_source_concept_name.value_counts(dropna=False),
      df2.xtn_race_ethnicity_source_concept_name.value_counts(dropna=False),
      df2.merged_race_ethnicity_v1.value_counts(dropna=False),
      df2.merged_race_ethnicity_v2.value_counts(dropna=False),
      df2.gender_source_concept_name.value_counts(dropna=False),
      df2.xtn_gender_identity_source_concept_name.value_counts(dropna=False),
      df2.merged_gender.value_counts(dropna=False),
      df2.xtn_sexual_orientation_source_concept_name.value_counts(dropna=False))

print(df2.current_age.median(),
      df2.current_age.max(),
      df2.current_age.min(),
      df2.current_age.isna().sum())

print(df2.xtn_five_digit_zip.value_counts(dropna=False))

In [25]:
patient_data = everything[['person_id', 'xtn_patient_epic_mrn', 'visit_start_datetime', 'xtn_birth_date',
                           'xtn_race_ethnicity_source_concept_name', 'race_source_concept_name','merged_race_ethnicity_v2', 'gender_source_concept_name', 'xtn_gender_identity_source_concept_name', 'merged_gender', 'xtn_sexual_orientation_source_concept_name', 'xtn_five_digit_zip']]

# dup = ['person_id', 'xtn_patient_epic_mrn', 'xtn_birth_date', 'xtn_race_ethnicity_source_concept_code',
#              'xtn_race_ethnicity_source_concept_name', 'race_source_concept_code', 'race_source_concept_name', 'merged_race_ethnicity',
#              'gender_source_concept_code', 'gender_source_concept_name', 'xtn_gender_identity_source_concept_code', 'merged_gender',
#              'xtn_gender_identity_source_concept_name', 'xtn_sexual_orientation_source_concept_code', 'xtn_sexual_orientation_source_concept_name']

patient_data = patient_data.sort_values(by=['visit_start_datetime'], ascending=[False])
dup = ['person_id']
patient_data = patient_data.drop_duplicates(subset=dup, keep='first')
print(patient_data.nunique(axis=0), patient_data.person_id.value_counts(dropna=False))
patient_data

person_id                                     466692
xtn_patient_epic_mrn                          466692
visit_start_datetime                           85419
xtn_birth_date                                 38777
xtn_race_ethnicity_source_concept_name             7
race_source_concept_name                         100
merged_race_ethnicity_v2                          52
gender_source_concept_name                         6
xtn_gender_identity_source_concept_name            9
merged_gender                                     12
xtn_sexual_orientation_source_concept_name         7
xtn_five_digit_zip                              7274
dtype: int64 2332301     1
12894950    1
3137073     1
3052321     1
3985891     1
           ..
2415046     1
4584300     1
3495217     1
2322891     1
6776528     1
Name: person_id, Length: 466692, dtype: int64


,person_id,xtn_patient_epic_mrn,visit_start_datetime,xtn_birth_date,xtn_race_ethnicity_source_concept_name,race_source_concept_name,merged_race_ethnicity_v2,gender_source_concept_name,xtn_gender_identity_source_concept_name,merged_gender,xtn_sexual_orientation_source_concept_name,xtn_five_digit_zip
5942682,2332301,2792028,2023-03-24 09:30:00,1963-03-22,Asian,CHINESE,Asian,Male,Male,Male,Lesbian or Gay,10025.0
27500514,15033805,7929561,2023-03-10 09:30:00,1967-11-13,Black or African American,Mixed racial group,Black or African American,Female,Female,Female,Straight (not lesbian or gay),10128.0
28323114,15883069,K153862,2023-03-08 00:00:00,1992-12-29,Hispanic or Latino,OTHER,Hispanic or Latino,Female,NaN,Female,NaN,11373.0
1339306,1327337,E516395,2023-02-23 11:30:00,1975-02-09,Hispanic or Latino,OTHER,Hispanic or Latino,Male,Male,Male,NaN,10456.0
10507378,3461222,8411899,2023-02-23 09:00:00,1951-10-07,Hispanic or Latino,OTHER,Hispanic or Latino,Male,Male,Male,NaN,10459.0
...,...,...,...,...,...,...,...,...,...,...,...,...
14240017,5851229,2528016,2006-07-13,1943-05-01,White,WHITE,White,Male,NaN,Male,NaN,85338.0
10246392,3418342,2579506,2006-07-11,1914-12-12,White,WHITE,White,Female,NaN,Female,NaN,10028.0
22682656,14212808,1862429,2006-07-11,1925-08-12,White,WHITE,White,Female,NaN,Female,NaN,10024.0
9952788,3276160,2522394,2006-07-05,1934-04-08,NaN,Hispanic/Latino,Hispanic/Latino,Female,NaN,Female,NaN,10036.0


In [26]:
patient_data['xtn_birth_date'] = pd.to_datetime(patient_data['xtn_birth_date'])
def calculate_age(born):
    year = born.year
    today = datetime.today().year
    return today - year # today.year - born.year - ((today.month, today.day) < (born.month, born.day))

patient_data['current_age'] = patient_data['xtn_birth_date'].apply(calculate_age)
#

In [27]:
# df2 = pd.read_csv("../data/clean_psych_patient.csv")

patient_data = patient_data.replace(['Indeterminate', 'Not recorded on birth certificate'], np.nan)
patient_data = patient_data.replace(['AMERICAN INDIAN OR ALASKAN', 'Hispanic/Latino', 'Hispanic', 'AFRICAN-AMERICAN', 'OTHER', 'PATIENT DECLINED', 'WHITE', 'ASIAN INDIAN', 'CHINESE',
                   'KOREAN', 'CHINESE AMERICAN', 'VIETNAMESE', 'OTHER: NORTH AFRICAN', 'JAPANESE', 'NATIVE HAWAIIAN', 'OTHER PACIFIC ISLANDER', 'THAI',
                   'PAPUA NEW GUINEAN', 'GUINEAN', 'FILIPINO', 'FILIPINO AMERICAN', 'BANGLADESHI', 'UGANDAN', 'PAKISTANI', 'WEST INDIAN', 'TRINIDADIAN', 'JAMAICAN', 'HAITIAN', 'TAIWANESE', 'NIGERIAN', 'ETHIOPIAN', 'DOMINICA ISLANDER', 'GRENADIAN', 'MADAGASCAR', 'POLYNESIAN', 'SRI LANKAN', 'INDONESIAN', 'MALIAN', 'GHANAIAN', 'BARBADIAN', 'TAHITIAN', 'ST VINCENTIAN', 'TOKELAUAN'],
                  ['American Indian or Alaska Native', 'Hispanic or Latino', 'Hispanic or Latino', 'Black or African American', 'Other Race', 'Decline to Answer', 'White', 'Asian', 'Asian', 'Asian', 'Asian', 'Asian', 'Black or African American', 'Asian', 'Native Hawaiian or Other Pacific Islander', 'Native Hawaiian or Other Pacific Islander', 'Asian', 'Native Hawaiian or Other Pacific Islander', 'Native Hawaiian or Other Pacific Islander', 'Native Hawaiian or Other Pacific Islander','Native Hawaiian or Other Pacific Islander', 'Asian', 'Black or African American', 'Asian', 'Asian', 'Mixed racial group', 'Black or African American', 'Black or African American', 'Asian', 'Black or African American', 'Black or African American', 'Black or African American', 'Black or African American', 'Black or African American', 'Native Hawaiian or Other Pacific Islander', 'Asian', 'Asian', 'Black or African American', 'Black or African American', 'Black or African American', 'Native Hawaiian or Other Pacific Islander', 'Black or African American', 'Native Hawaiian or Other Pacific Islander'])

print(patient_data.merged_race_ethnicity_v2.value_counts(dropna=False),
      patient_data.merged_gender.value_counts(dropna=False),
      patient_data.xtn_sexual_orientation_source_concept_name.value_counts(dropna=False))

# print(patient_data.xtn_five_digit_zip.value_counts(dropna=False))

White                                        189646
NaN                                           88300
Hispanic or Latino                            63363
Black or African American                     55604
Other Race                                    47389
Asian                                         16992
Mixed racial group                             2784
Native Hawaiian or Other Pacific Islander      1346
Decline to Answer                               839
American Indian or Alaska Native                429
Name: merged_race_ethnicity_v2, dtype: int64 Female                                 261312
Male                                   201141
Non-binary                               1253
Transgender Female / Male-to-Female      1015
Transgender Male / Female-to-Male         694
Choose not to disclose                    429
Gender Non-conforming                     250
Gender Queer                              226
Other                                     221
NaN                  

In [29]:
def age(row):
    if row >99:
        return 99
    else:
        return row

patient_data['current_age'] =patient_data.current_age.apply(age)

In [30]:
print(patient_data.current_age.mean(),
      patient_data.current_age.max(),
      patient_data.current_age.min(),
      patient_data.current_age.isna().sum())


49.579769955345284 99 0 0


In [34]:
pd.DataFrame.to_csv(patient_data.sort_values(by=['person_id'], ascending=[True]), "../data/clean_psych_patient.csv",index = False)
# most updated version 11/3/22

In [3]:
# psych_encounters.xtn_birth_date.clip()
#
# psych_encounters.clip(lower=pd.Series({'xtn_birth_date': 2.5}), upper=pd.Series({'xtn_birth_date': 2.5}), axis=1)

all_encounters['visit_year'] = pd.to_datetime(all_encounters['visit_start_datetime']).dt.year
all_encounters['birth_year'] = pd.to_datetime(all_encounters['xtn_birth_date']).dt.year

all_encounters['age_at_visit'] = all_encounters['visit_year'] - all_encounters['birth_year']

# def calculate_age_diagnosis(born):
#     birth_year = born.year
#     visit_year = all_encounters.visit_start_datetime.year
#     return birth_year - visit_year # today.year - born.year - ((today.month, today.day) < (born.month, born.day))
#
# all_encounters['age_diagnosis'] = all_encounters['xtn_birth_date'].apply(calculate_age_diagnosis)
all_encounters.head(20)

,person_id,visit_occurrence_id,etl_epic_encounter_key,xtn_department_name,xtn_visit_type_source_concept_id,xtn_visit_type_source_concept_code,visit_start_datetime,visit_end_datetime,xtn_patient_epic_mrn,xtn_birth_date,...,insurance_payor_class,insurance_status,ICD_name,ICD_code,ICD_code_h,ICD_code_top_h,CCSR_code,visit_year,birth_year,age_at_visit
0,845,172058493,285636709,10 UNION SQ E RESP,2000000112,Outpatient Visit,2022-01-25 00:00:00,2022-01-25 00:00:00,3754282,1974-09-28,...,Medicaid Managed Care,Public,NaN,NaN,NaN,NaN,NaN,2022,1974,48
1,845,172058493,285636709,10 UNION SQ E RESP,2000000112,Outpatient Visit,2022-01-25 00:00:00,2022-01-25 00:00:00,3754282,1974-09-28,...,Medicaid Managed Care,Public,NaN,NaN,NaN,NaN,NaN,2022,1974,48
2,845,172058493,285636709,10 UNION SQ E RESP,2000000112,Outpatient Visit,2022-01-25 00:00:00,2022-01-25 00:00:00,3754282,1974-09-28,...,Medicaid Managed Care,Public,NaN,NaN,NaN,NaN,NaN,2022,1974,48
3,845,185133868,291089587,10 UNION SQ PFT,2000000112,Outpatient Visit,2022-02-22 00:00:00,2022-02-22 00:00:00,3754282,1974-09-28,...,Medicaid Managed Care,Public,NaN,NaN,NaN,NaN,NaN,2022,1974,48
4,845,185133868,291089587,10 UNION SQ PFT,2000000112,Outpatient Visit,2022-02-22 00:00:00,2022-02-22 00:00:00,3754282,1974-09-28,...,Medicaid Managed Care,Public,NaN,NaN,NaN,NaN,NaN,2022,1974,48
5,845,185133868,291089587,10 UNION SQ PFT,2000000112,Outpatient Visit,2022-02-22 00:00:00,2022-02-22 00:00:00,3754282,1974-09-28,...,Medicaid Managed Care,Public,NaN,NaN,NaN,NaN,NaN,2022,1974,48
6,845,194027752,293787212,17 E 102ND RHEUMATOLOGY CLINIC,2000000112,Outpatient Visit,2022-06-16 00:00:00,2022-06-16 00:00:00,3754282,1974-09-28,...,Medicaid Managed Care,Public,NaN,NaN,NaN,NaN,NaN,2022,1974,48
7,845,194027752,293787212,17 E 102ND RHEUMATOLOGY CLINIC,2000000112,Outpatient Visit,2022-06-16 00:00:00,2022-06-16 00:00:00,3754282,1974-09-28,...,Medicaid Managed Care,Public,NaN,NaN,NaN,NaN,NaN,2022,1974,48
8,845,194027752,293787212,17 E 102ND RHEUMATOLOGY CLINIC,2000000112,Outpatient Visit,2022-06-16 00:00:00,2022-06-16 00:00:00,3754282,1974-09-28,...,Medicaid Managed Care,Public,NaN,NaN,NaN,NaN,NaN,2022,1974,48
9,845,197004100,302037736,17 E 102ND RHEUMATOLOGY CLINIC,2000000112,Outpatient Visit,2022-07-05 00:00:00,2022-07-05 00:00:00,3754282,1974-09-28,...,Medicaid Managed Care,Public,NaN,NaN,NaN,NaN,NaN,2022,1974,48


In [6]:
all_encounters.merged_race_ethnicity_v2.value_counts(dropna=False)

White                                        12387606
Hispanic or Latino                            6122720
Black or African American                     4301307
Other Race                                    2526292
NaN                                           1854888
Asian                                          962550
Mixed racial group                              69772
Native Hawaiian or Other Pacific Islander       45890
Decline to Answer                               35075
American Indian or Alaska Native                20141
Name: merged_race_ethnicity_v2, dtype: int64

In [5]:
psych_encounters = all_encounters.groupby('visit_occurrence_id').filter(lambda x: x['psych_condition'].mean() > 0)
pd.DataFrame.to_csv(psych_encounters, "/Users/valena17/psych_nlp/data/clean_psych_encounters_conditions_patients.csv",index = False)